In [ ]:
!git clone https://github.com/matterport/Mask_RCNN.git
%cd Mask_RCNN
!python setup.py install
!pip install -qr requirements.txt
!pip uninstall keras -y
!pip uninstall keras-nightly -y
!pip uninstall keras-Preprocessing -y
!pip uninstall keras-vis -y
!pip uninstall tensorflow -y
!pip uninstall h5py -y
!pip uninstall scikit-image -y
!pip install tensorflow==1.13.1
!pip install keras==2.0.8
!pip install h5py==2.10.0
!pip install scikit-image==0.16.2

Cloning into 'Mask_RCNN'...
remote: Enumerating objects: 956, done.
remote: Total 956 (delta 0), reused 0 (delta 0), pack-reused 956
Receiving objects: 100% (956/956), 125.23 MiB | 51.87 MiB/s, done.
Resolving deltas: 100% (565/565), done.
/content/Mask_RCNN
/usr/local/lib/python3.7/dist-packages/setuptools/dist.py:700: UserWarning: Usage of dash-separated 'description-file' will not be supported in future versions. Please use the underscore name 'description_file' instead
  % (opt, underscore_opt))
/usr/local/lib/python3.7/dist-packages/setuptools/dist.py:700: UserWarning: Usage of dash-separated 'license-file' will not be supported in future versions. Please use the underscore name 'license_file' instead
  % (opt, underscore_opt))
/usr/local/lib/python3.7/dist-packages/setuptools/dist.py:700: UserWarning: Usage of dash-separated 'requirements-file' will not be supported in future versions. Please use the underscore name 'requirements_file' instead
  % (opt, underscore_opt))
running i

In [ ]:
import os
from collections import defaultdict
from os import listdir
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from mrcnn.config import Config
from mrcnn.model import MaskRCNN
from mrcnn.utils import Dataset
from mrcnn.utils import compute_ap
from mrcnn.model import load_image_gt
from mrcnn.model import mold_image

/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

In [ ]:
class TinyHandDataset(Dataset):
    def load_dataset(self, is_train = False, is_val = False):
        base_dir = '/content/drive/MyDrive/Colab Notebooks/resized_and_labelled_pictures/'
        self.add_class("dataset", 0, "tiny_hand")
        self.add_class("dataset", 1, "normal_hand")
        if is_train:
            images_dir = base_dir + 'images/train'
            labels_dir = base_dir + 'labels/train'
        elif is_val:
            images_dir = base_dir + 'images/val'
            labels_dir = base_dir + 'labels/val'
        else:
            images_dir = base_dir + 'images/test'
            labels_dir = base_dir + 'labels/test'
        for filename in listdir(images_dir):
            image_id = filename[:-4]
            img_path = os.path.join(images_dir, filename)
            label_path = os.path.join(labels_dir, image_id + '.txt')
            self.add_image('dataset', 
                            image_id = image_id, 
                            path = img_path, 
                            annotation = label_path)
      
    def extract_boxes(self, filename):
        with open(filename, "r") as file:
            annotations = file.read().split('\n')
            file.close()
        annotations = [i.split(' ') for i in annotations if i != '']
        boxes = defaultdict(list)
        for i in annotations:
            label = int(i[0])
            x_min = int(i[1])
            y_min = int(i[2])
            x_max = int(i[3])
            y_max = int(i[4])
            boxes[label].append([x_min, y_min, x_max, y_max])
        return boxes
    
    def load_mask(self, image_id, height = 1024, width = 1024):
        info = self.image_info[image_id]
        path = info['annotation']
        boxes = self.extract_boxes(path)
        cntr = 1
        for v in boxes.values():
            for v_2 in v:
                cntr += 1   
        masks = np.zeros([height, width, cntr - 1], dtype = 'uint8')
        class_ids = list()  
        for idx, (k, v) in enumerate(boxes.items()):
            for v_2 in v:
                row_s, row_e = v_2[1], v_2[3]
                col_s, col_e = v_2[0], v_2[2]
                masks[row_s:row_e, col_s:col_e, idx] = 1
                if k == 0:
                    class_ids.append(self.class_names.index('tiny_hand'))
                elif k == 1:
                    class_ids.append(self.class_names.index('normal_hand'))
        return masks, np.asarray(class_ids, dtype = 'int32')
  
    def image_reference(self, image_id):
        info = self.image_info[image_id]
        return info['path']

In [ ]:
class OneTinyHandConfig(Config):
    NAME = "onetinyhand_cfg"
    NUM_CLASSES = 1 + 2
    STEPS_PER_EPOCH = 400
    VALIDATION_STEPS = 5
    BATCH_SIZE = 2

In [ ]:
class PredictionConfig(Config):
    NAME = "prediction_cfg"
    NUM_CLASSES = 1 + 2
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

In [ ]:
def evaluate_model(dataset, model, cfg):
    ls_of_average_precisions = list()
    for image_id in dataset.image_ids:
        image, image_meta, ground_truth_class_id, ground_truth_bbox, ground_truth_mask = (
              load_image_gt(dataset, cfg, image_id, use_mini_mask = False)
        )
        scaled_image = mold_image(image, cfg)
        sample = np.expand_dims(scaled_image, 0)
        yhat = model.detect(sample, verbose = 0)
        r = yhat[0]
        average_precision, _, _, _ = (
              compute_ap(ground_truth_bbox, 
                         ground_truth_class_id, 
                         ground_truth_mask, 
                         r["rois"], 
                         r["class_ids"], 
                         r["scores"], 
                         r["masks"]))
        ls_of_average_precisions.append(average_precision)
    mean_average_precision = np.mean(ls_of_average_precisions)
    return mean_average_precision

In [ ]:
def plot_actual_vs_predicted(dataset, model, cfg, n_images = 5):
    for i in range(n_images):
        image = dataset.load_image(i)
        mask, _ = dataset.load_mask(i)
        scaled_image = mold_image(image, cfg)
        sample = np.expand_dims(scaled_image, 0)
        yhat = model.detect(sample, verbose = 0)[0]
        plt.figure(figsize = (12, 12), dpi = 300)
        plt.subplot(n_images, 2, i*2 + 1)
        plt.imshow(image)
        plt.title('Actual')
        for j in range(mask.shape[2]):
            plt.imshow(mask[:, :, j], cmap = 'Blues', alpha = 0.5)
        plt.subplot(n_images, 2, i*2 + 2)
        plt.imshow(image)
        plt.title('Predicted')
        ax = plt.gca()
        for i in yhat['rois']:
            min_y, min_x, max_y, max_x = i
            width, height = max_x - min_x, max_y - min_y
            rect = Rectangle((min_x, min_y), 
                    width, 
                    height, 
                    fill = False, 
                    color = 'red')
            ax.add_patch(rect)
    plt.show()

In [ ]:
train_set = TinyHandDataset()
train_set.load_dataset(is_train=True)
train_set.prepare()
print('Train: %d' % len(train_set.image_ids))

Train: 415


In [ ]:
val_set = TinyHandDataset()
val_set.load_dataset(is_val = True)
val_set.prepare()
print('Validation: %d' % len(val_set.image_ids))

Validation: 74


In [ ]:
test_set = TinyHandDataset()
test_set.load_dataset()
test_set.prepare()
print('Test: %d' % len(test_set.image_ids))

Test: 87


In [ ]:
# config = OneTinyHandConfig()
# model = MaskRCNN(mode = 'training', 
#                  model_dir = '/content/drive/MyDrive/Colab Notebooks/maskRCNN models', 
#                  config = config)
# model.load_weights('/content/drive/MyDrive/Colab Notebooks/mask_rcnn_coco.h5', 
#                    by_name = True, 
#                    exclude = ["mrcnn_class_logits", 
#                               "mrcnn_bbox_fc",  
#                               "mrcnn_bbox", 
#                               "mrcnn_mask"])
# model.train(train_set, 
#             val_set, 
#             learning_rate = config.LEARNING_RATE, 
#             epochs = 5, 
#             layers = 'heads')

In [ ]:
cfg = PredictionConfig()
model = MaskRCNN(mode = 'inference', 
                 model_dir = '/content/drive/MyDrive/Colab Notebooks/maskRCNN models', 
                 config = cfg)
model_path = "/content/drive/MyDrive/Colab Notebooks/maskRCNN models/onetinyhand_cfg20220612T1718/mask_rcnn_onetinyhand_cfg_0004.h5"
model.load_weights(model_path, by_name = True)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
Use tf.cast instead.
Re-starting from epoch 4


In [ ]:
plot_actual_vs_predicted(train_set, model, cfg)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
plot_actual_vs_predicted(test_set, model, cfg)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
train_mAP = evaluate_model(train_set, model, cfg)
print("Train mAP: %.3f" % train_mAP)

Train mAP: 0.615


In [ ]:
test_mAP = evaluate_model(test_set, model, cfg)
print("Test mAP: %.3f" % test_mAP)

Test mAP: 0.403
